# SVM - Climate Sentiment Multiclass Classification
## CS522 Project

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC
from Common.DataCenter import data_center
from Common.preprocessor import normalize_preprocessing
from Common.UtilFuncs import print_evaluation

%matplotlib inline

### Text preprocessing

In [2]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    
    # preprocessing with traditional NLP methodology
    X_train_normalized = normalize_preprocessing(X_train)
    X_test_normalized  = normalize_preprocessing(X_test)
    
    # vectorization
    # Convert texts to vectors by TFIDF
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    X_train_vec  = vectorizer.fit_transform(X_train_normalized)
    X_test_vec   = vectorizer.transform(X_test_normalized)
      
    return X_train_vec, X_test_vec

### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


### Run SVM

In [4]:
# parameter:  vectorised X and encoded y of training set and test set
def run_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight='balanced'), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    y_pred          = SVM.predict(X_test_vec)
    return  y_pred


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # Run SVM and evaluate the results
    y_pred = run_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Print the evaluation
    print_evaluation(y_test_vec, y_pred, labels=[0,1,2,3])


### Print the distribution of labels

In [ ]:
def print_distribution(hint, y):
    df = data_center.df((y, y))
    c = df['sentiment'].value_counts(sort = False)
    l = len(df)
    print("%s: %s" % (hint, ("%.1f%%, "*(len(c)-1)+"%.1f%%") % tuple([x*100/l for x in list(c)])))

### Main entry

**Load the database and split it into training set, test set, noisy set, validation set**

In [6]:
dc = data_center("twitter_sentiment_data_clean.csv", test_size=4000, noisy_size=3000, validation_size=0)

print("####################################################")
print("Total data size: ",       dc.get_len())
print("Total train data size: ", dc.get_train_len())
print("Total test data size: ",  dc.get_test_len())

####################################################
Total data size:  41033
Total train data size:  20033
Total test data size:  8000


**Get the test set for evaluation**

In [7]:
X_test, y_test = dc.get_test()


**Set distributions of training set.**

In [ ]:
# distribution of training set
train_distribution = None


**Prepare the noisy set.**

In [ ]:
# distribution of external noisy
external_noisy_distribution = [0.25, 0.25, 0.25, 0.25]
lstNoisyInfo = [("mislabeled",dc.get_noisy_len())]
print("Noisy set size is %d"                % dc.get_noisy_len())

# add the external noisy data (irrelevant texts)
added_size = dc.add_noisy(noisy_source="irrelevant",
                          distribution = external_noisy_distribution, size = 6000) # max size: 34259
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("irrelevant",added_size))

# add the external noisy data (translated texts)
added_size = dc.add_noisy(noisy_source="translated",
                          distribution = "reserve_labels", size = 6000) # max size: 6146
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("translated",added_size))

print("Noisy set new size is %d"                % dc.get_noisy_len())



**Run experiments with different training sets, and use the same test set.**

In [8]:
print("-------------- No noisy training sets ----------")
for size in [2000, 4000, 5000, 8000, 10000, 15000, 20000]:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size, train_distribution)
    print("* Training set size: %d samples: " % (len(X_train)))
    print_distribution("  Sentiment distribution", y_train)

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

print("-------------- Noisy training sets -------------")
print("The proportions of the noise sources %s: " % [x[0] for x in lstNoisyInfo],
      [round(x[1]*100/dc.get_noisy_len(),1) for x in lstNoisyInfo])
for size in [(4000, 1000), (8000, 3000), (15000, 5000)]:
    # Get a noisy training set
    X_train, y_train = dc.get_train_with_noisy(size[0], size[1], train_distribution)
    print("* Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))
    print_distribution("  Sentiment distribution", y_train)

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

-----------------------------------------------
Training set size: 1000 samples (5.0%): 


TypeError: fit() got an unexpected keyword argument 'n_classes'